In [ ]:
import hashlib
import os

# 다운로드할 MNIST 데이터셋 URL과 MD5 해시값
urls = [
    ('https://ossci-datasets.s3.amazonaws.com/mnist/train-images-idx3-ubyte.gz', 'f68b3c2dcbeaaa9fbdd348bbdeb94873'),
    ('https://ossci-datasets.s3.amazonaws.com/mnist/train-labels-idx1-ubyte.gz', 'd53e105ee54ea40749a09fcbcd1e9432'),
    ('https://ossci-datasets.s3.amazonaws.com/mnist/t10k-images-idx3-ubyte.gz', '9fb629c4189551a2d022fa330f9573f3'),
    ('https://ossci-datasets.s3.amazonaws.com/mnist/t10k-labels-idx1-ubyte.gz', 'ec29112dd5afa0611ce80d1b7f02629c')
]

# 파일 다운로드 함수
def download_and_verify(url, md5_hash):
    filename = url.split("/")[-1]
    # 파일 다운로드
    os.system(f"wget {url}")

    # 다운로드된 파일의 MD5 해시값 확인
    downloaded_md5 = hashlib.md5(open(filename, 'rb').read()).hexdigest()

    # MD5 해시값 검증
    if downloaded_md5 == md5_hash:
        print(f"{filename} 다운로드 성공! 해시값이 일치합니다.")
    else:
        print(f"{filename} 다운로드 실패! 해시값이 일치하지 않습니다.")

# 각 파일에 대해 다운로드 및 해시값 검증
for url, md5 in urls:
    download_and_verify(url, md5)

train-images-idx3-ubyte.gz 다운로드 성공! 해시값이 일치합니다.
train-labels-idx1-ubyte.gz 다운로드 성공! 해시값이 일치합니다.
t10k-images-idx3-ubyte.gz 다운로드 성공! 해시값이 일치합니다.
t10k-labels-idx1-ubyte.gz 다운로드 성공! 해시값이 일치합니다.


In [ ]:
import numpy as np
import gzip
import struct

# 파일 압축 해제 함수
def extract_gz(filename):
    with gzip.open(filename, 'rb') as f_in:
        with open(filename[:-3], 'wb') as f_out:
            f_out.write(f_in.read())

# idx 파일 읽기 함수
def read_idx(filename):
    with open(filename, 'rb') as f:
        magic_number = struct.unpack('>I', f.read(4))[0]
        if magic_number == 2051:  # 이미지 파일
            num_images = struct.unpack('>I', f.read(4))[0]
            rows = struct.unpack('>I', f.read(4))[0]
            cols = struct.unpack('>I', f.read(4))[0]
            images = np.frombuffer(f.read(), dtype=np.uint8).reshape(num_images, rows, cols)
            return images
        elif magic_number == 2049:  # 레이블 파일
            num_labels = struct.unpack('>I', f.read(4))[0]
            labels = np.frombuffer(f.read(), dtype=np.uint8)
            return labels

# MNIST 데이터 로드
train_images_path = 'train-images-idx3-ubyte.gz'
train_labels_path = 'train-labels-idx1-ubyte.gz'
test_images_path = 't10k-images-idx3-ubyte.gz'
test_labels_path = 't10k-labels-idx1-ubyte.gz'

extract_gz(train_images_path)
extract_gz(train_labels_path)
extract_gz(test_images_path)
extract_gz(test_labels_path)

x_train = read_idx('train-images-idx3-ubyte') / 255.0
y_train = read_idx('train-labels-idx1-ubyte')
x_test = read_idx('t10k-images-idx3-ubyte') / 255.0
y_test = read_idx('t10k-labels-idx1-ubyte')

x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

# 합성곱 연산

def conv2d(input, kernel):
    input_h, input_w, input_c = input.shape
    kernel_h, kernel_w, _, output_c = kernel.shape
    output_h, output_w = input_h - kernel_h + 1, input_w - kernel_w + 1
    output = np.zeros((output_h, output_w, output_c))

    for h in range(output_h):
        for w in range(output_w):
            for c in range(output_c):
                output[h, w, c] = np.sum(input[h:h+kernel_h, w:w+kernel_w, :] * kernel[:, :, :, c])
    return output

# 맥스 풀링

def max_pooling(input, size=2):
    input_h, input_w, input_c = input.shape
    output_h, output_w = input_h // size, input_w // size
    output = np.zeros((output_h, output_w, input_c))

    for h in range(output_h):
        for w in range(output_w):
            for c in range(input_c):
                output[h, w, c] = np.max(input[h*size:(h+1)*size, w*size:(w+1)*size, c])
    return output

# 활성화 함수 및 손실 함수

def relu(x):
    return np.maximum(0, x)

def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

def cross_entropy_loss(y_pred, y_true):
    y_one_hot = np.eye(10)[y_true]
    return -np.sum(y_one_hot * np.log(y_pred + 1e-9)) / len(y_true)

# 가중치 업데이트

def update_weights(W, b, dW, db, lr=0.01):
    W -= lr * dW
    b -= lr * db.squeeze()  # 차원 맞추기
    return W, b

# 학습 함수

def train_cnn(x_train, y_train, x_test, y_test, epochs=5, lr=0.01):
    kernel = np.random.randn(3, 3, 1, 2) * np.sqrt(2 / (3 * 3 * 1))
    flattened_size = ((28 - 3 + 1) // 2) ** 2 * 2
    W = np.random.randn(flattened_size, 10) * np.sqrt(2 / flattened_size)
    b = np.zeros(10)

    for epoch in range(epochs):
        loss, correct = 0, 0
        for i in range(len(x_train)):
            img, label = x_train[i], y_train[i]
            conv_out = relu(conv2d(img, kernel))
            pooled_out = max_pooling(conv_out)
            flattened = pooled_out.flatten()
            logits = np.dot(flattened, W) + b
            y_pred = softmax(logits.reshape(1, -1))
            loss += cross_entropy_loss(y_pred, np.array([label]))

            if np.argmax(y_pred) == label:
                correct += 1

            y_one_hot = np.eye(10)[label]
            dL_dlogits = y_pred - y_one_hot
            dW = np.outer(flattened, dL_dlogits)
            db = dL_dlogits
            W, b = update_weights(W, b, dW, db, lr)

        print(f"Epoch {epoch+1}: Loss={loss:.4f}, Accuracy={(correct / len(x_train)) * 100:.2f}%")

    # 테스트 평가
    correct = 0
    for i in range(len(x_test)):
        img, label = x_test[i], y_test[i]
        conv_out = relu(conv2d(img, kernel))
        pooled_out = max_pooling(conv_out)
        flattened = pooled_out.flatten()
        logits = np.dot(flattened, W) + b
        y_pred = softmax(logits.reshape(1, -1))

        if np.argmax(y_pred) == label:
            correct += 1

    print(f"Test Accuracy: {(correct / len(x_test)) * 100:.2f}%")

train_cnn(x_train, y_train, x_test, y_test, epochs=5, lr=0.01)


Epoch 1: Loss=19928.0321, Accuracy=90.31%
Epoch 2: Loss=15293.5647, Accuracy=92.46%
Epoch 3: Loss=14107.4863, Accuracy=93.06%
Epoch 4: Loss=13411.8178, Accuracy=93.38%
Epoch 5: Loss=12937.7804, Accuracy=93.61%
Test Accuracy: 93.04%
